In [1]:
import torch
import transformers
transformers.set_seed(42)
device = "cuda"

In [2]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa").to(device)
# model = BertForMaskedLM.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa").to(device)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [3]:
model.config.num_attention_heads

12

In [6]:
model.config.is_decoder = True # this was super important

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1")

In [9]:
def ltrattn(shape):
    mask = torch.full(shape,1)
    return torch.tril(mask, diagonal=-1)

def rtlattn(shape):
    mask = torch.full(shape,1)
    return torch.triu(mask, diagonal=1)

print(ltrattn((5,5)))
print(rtlattn((6,6)))


tensor([[0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0]])
tensor([[0, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0]])


In [10]:
train_ds = ds["train"]
inputs = tokenizer(train_ds[10]["text"], return_tensors="pt", padding='max_length', truncation=True)

inputs["input_ids"].size()

torch.Size([1, 512])

In [59]:
train_ds[5]["text"]

" It met with positive sales in Japan , and was praised by both Japanese and western critics . After release , it received downloadable content , along with an expanded edition in November of that year . It was also adapted into manga and an original video animation series . Due to low sales of Valkyria Chronicles II , Valkyria Chronicles III was not localized , but a fan translation compatible with the game 's expanded edition was released in 2014 . Media.Vision would return to the franchise with the development of Valkyria : Azure Revolution for the PlayStation 4 . \n"

In [55]:
train_ds[10]["text"]

' The game \'s battle system , the <unk> system , is carried over directly from <unk> Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a character

In [13]:
output = model(**{k: v.to(device) for k, v in inputs.items()}, encoder_attention_mask=rtlattn(inputs["input_ids"].size() + (inputs["input_ids"].size(1),)))

> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(1469)forward()
   1468         ipdb.set_trace()
-> 1469         outputs = self.bert(
   1470             input_ids,



ipdb>  attention_mask.size()


torch.Size([1, 512])


ipdb>  p encoder_attention_mask.size()


torch.Size([1, 512, 512])


...

ipdb>  n


> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(1049)forward()
   1048         output_hidden_states = (
-> 1049             output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
   1050         )



ipdb>  n


> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(1048)forward()
   1047         output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
-> 1048         output_hidden_states = (
   1049             output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states



ipdb>  q


In [140]:
output.logits

tensor([[[-6.3281, -6.3555, -6.4531,  ..., -5.5234, -4.1797, -5.7891],
         [-6.7891, -6.6914, -6.7812,  ..., -6.1680, -5.1094, -5.5273],
         [-7.1641, -7.1055, -7.0625,  ..., -6.2383, -5.3711, -5.5273],
         ...,
         [-8.3516, -8.4375, -8.3516,  ..., -7.6289, -7.0078, -5.6016],
         [-7.7617, -7.8789, -7.7695,  ..., -7.0938, -6.7461, -5.0430],
         [-7.6602, -7.7500, -7.6953,  ..., -6.9492, -6.4766, -4.9531]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>)

In [11]:
output2 = model(**{k: v.to(device) for k, v in inputs.items()}, encoder_attention_mask=ltrattn(inputs["input_ids"].size() + (inputs["input_ids"].size(1),)))

> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(1469)forward()
   1468         ipdb.set_trace()
-> 1469         outputs = self.bert(
   1470             input_ids,



ipdb>  c


> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(444)forward()
    443         ipdb.set_trace()
--> 444         attn_output = torch.nn.functional.scaled_dot_product_attention(
    445             query_layer,



ipdb>  l


    439         is_causal = (
    440             True if self.is_decoder and not is_cross_attention and attention_mask is None and tgt_len > 1 else False
    441         )
    442 
    443         ipdb.set_trace()
--> 444         attn_output = torch.nn.functional.scaled_dot_product_attention(
    445             query_layer,
    446             key_layer,
    447             value_layer,
    448             attn_mask=attention_mask,
    449             dropout_p=self.dropout_prob if self.training else 0.0,



...

ipdb>  attention_mask


tensor([[[[     0., -65504., -65504.,  ..., -65504., -65504., -65504.],
          [     0.,      0., -65504.,  ..., -65504., -65504., -65504.],
          [     0.,      0.,      0.,  ..., -65504., -65504., -65504.],
          ...,
          [     0.,      0.,      0.,  ..., -65504., -65504., -65504.],
          [     0.,      0.,      0.,  ..., -65504., -65504., -65504.],
          [     0.,      0.,      0.,  ..., -65504., -65504., -65504.]]]],
       device='cuda:0', dtype=torch.float16)


ipdb>  attention_mask.size()


torch.Size([1, 1, 512, 512])
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /home/sipb/transformer-shortest-paths/NLP/transformers/src/transformers/models/bert/modeling_bert.py(444)forward()
    443         ipdb.set_trace()
--> 444         attn_output = torch.nn.functional.scaled_dot_product_attention(
    445             query_layer,



ipdb>  q
ipdb>  q


In [138]:
output2.logits

tensor([[[-6.3281, -6.3555, -6.4531,  ..., -5.5234, -4.1797, -5.7891],
         [-6.7891, -6.6914, -6.7812,  ..., -6.1680, -5.1094, -5.5273],
         [-7.1641, -7.1055, -7.0625,  ..., -6.2383, -5.3711, -5.5273],
         ...,
         [-8.3516, -8.4375, -8.3516,  ..., -7.6289, -7.0078, -5.6016],
         [-7.7617, -7.8789, -7.7695,  ..., -7.0938, -6.7461, -5.0430],
         [-7.6602, -7.7500, -7.6953,  ..., -6.9492, -6.4766, -4.9531]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>)

In [135]:
torch.equal(output.logits, output2.logits)

True

In [144]:
output2 = model(**{k: v.to(device) for k, v in inputs.items()}, encoder_attention_mask=torch.zeros(1, 512, 512))

In [145]:
output2.logits

tensor([[[ -8.8438,  -8.8750,  -8.7812,  ...,  -8.3672,  -8.1484,  -4.5195],
         [-12.5547, -12.2734, -12.4609,  ..., -11.4141, -10.2969,  -7.3320],
         [-12.8125, -12.8125, -12.7891,  ..., -12.1328, -10.5781,  -5.9453],
         ...,
         [ -7.9531,  -8.1797,  -8.2266,  ...,  -7.2188,  -6.5000,  -6.4688],
         [ -7.5234,  -7.7344,  -7.7305,  ...,  -6.7344,  -6.3359,  -6.0195],
         [ -7.8711,  -7.9453,  -8.0156,  ...,  -7.3555,  -7.1523,  -5.6680]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>)